// 创建索引
PUT /hotel
{
    "mappings":{
        "properties":{
            "name":{
                "type":"text"
            },
            "city":{
                "type":"keyword"
            },
            "price":{
                "type":"double"
            }
        }
    }
}

// 写入文档数据
// 可以不指定文档_id，改_id值由ES自动生成
POPST /hotel/_doc/001
{
    "name": "希尔顿酒店",
    "city": "shenzhen",
    "price": 99.99
}

// 根据_id搜索文档
GET /hotel/_doc/001

// 根据普通字段搜索文档
GET /hotel/_search
{
    "query": {
        "term": {
            "city": {
                "value": "shenzhen"
            }
        }
    }
}

// 根据文本字段搜索文档
GET /hotel/_search
{
    "query": {
        "match": {
            "name": "friend"
        }
    }
}

// 批量写入数据
PUT /hotel/_bulk
{"index":{}}
{"name":"x", "city":"x","price":99}

// 根据条件删除文档
POST /hotel/_delete_by_query
{
    "query": {
        "term": {
            "city": {
                "value": "三亚"
            }
        }
    }
}

// 删除索引
DELETE hotel

In [1]:
# 1. 连接ES

from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

In [2]:
# 2. 创建索引

es.indices.create(index="py_index01", ignore=400)

/var/folders/mq/fxmjb3ss6mg8hjxwd1sy_zbc0000gn/T/ipykernel_6834/544464998.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="py_index01", ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'py_index01'})

In [3]:
# 3. 插入数据

# 第1条数据

body = {
    "name": "tommy",
    "age": 20,
    "city": "shenzhen",
    "hobbies":"singing,dancing,reading"
}
es.index(index="py_index01", id=1, body=body)

ObjectApiResponse({'_index': 'py_index01', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [4]:
# 第2条数据

body = {
    "name": "sally",
    "age": 21,
    "city": "shenzhen",
    "hobbies": "jogging,watching,running"
}

es.index(index="py_index01", id=2, body=body)

ObjectApiResponse({'_index': 'py_index01', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

In [5]:
# 第3条数据

body = {
    "name" : "john",
    "age": 22,
    "city": "beijing",
    "hobbies": "liking,biking,jumping"
}

es.index(index="py_index01", id=3, body=body)

ObjectApiResponse({'_index': 'py_index01', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

In [6]:
# 4. match_all 查询所有

query = {
    "query": {
        "match_all": {}
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 110, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 1.0, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}, {'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}, {'_index': 'py_index01', '_id': '3', '_score': 1.0, '_source': {'name': 'john', 'age': 22, 'city': 'beijing', 'hobbies': 'liking,biking,jumping'}}]}}


In [7]:
# 5. 根据 term 查询

# term主要用于精确匹配哪些值，比如数字，日期，布尔值或 not_analyzed 的字符串(未经切词的文本数据类型)

query = {
    "query": {
        "term": {
            "age": 21
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 21, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}]}}


In [9]:
# 6. 根据 terms 查询

# terms 跟 term 有点类似，但 terms 允许指定多个匹配条件。 如果某个字段指定了多个值，那么文档需要一起去做匹配

# ① 根据age
query = {
    "query": {
        "terms": {
            "age": [20, 21]
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

query = {
    "query": {
        "terms": {
            "age": [21]
        }
    }
}

result_ = es.search(index="py_index01", body=query)

print(result_)

{'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 1.0, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}, {'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}]}}
{'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}]}}


In [10]:
# ② 根据name
query = {
    "query": {
        "terms": {
            "name": ["tommy", "sally"]
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 63, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 1.0, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}, {'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}]}}


In [11]:
# 7. range

# 按照指定范围查找一批数据:
'''
gt : 大于
gte : 大于等于
lt : 小于
lte : 小于等于
'''
query = {
    "query": {
        "range": {
            "age": {
                "gt": 20
            }
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}, {'_index': 'py_index01', '_id': '3', '_score': 1.0, '_source': {'name': 'john', 'age': 22, 'city': 'beijing', 'hobbies': 'liking,biking,jumping'}}]}}


In [12]:
# 8. exists / missing

# 查找文档中是否包含指定字段或没有某个字段，类似于SQL语句中的IS_NULL条件

query = {
    "query": {
        "exists": {
            "field": "name"
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 1.0, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}, {'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}, {'_index': 'py_index01', '_id': '3', '_score': 1.0, '_source': {'name': 'john', 'age': 22, 'city': 'beijing', 'hobbies': 'liking,biking,jumping'}}]}}


In [16]:
# 9. bool过滤

# 合并多个过滤条件查询结果的布尔逻辑
'''
must :: 多个查询条件的完全匹配,相当于 and。
must_not :: 多个查询条件的相反匹配，相当于 not。
should :: 至少有一个查询条件匹配, 相当于 or。
'''
query = {
    "query": {
        "bool": {
            "must": [
                {"term": {"name": "sally"}},
                {"term": {"age": 21}}

            ]
        }
    }
}
# query = {
    # "query": {
        # "bool": {
            # "must": [
                # {"term": {"name": "sally"}},
                # {"term": {"age": 21}}
            # ]
        # }
    # }
# }

result = es.search(index="py_index01", body=query)

print(result)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 2.2039728, 'hits': [{'_index': 'py_index01', '_id': '2', '_score': 2.2039728, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}]}}


In [17]:
# 案例2
'''
条件1：name != sally
条件2：字段age必须存在
条件3：age = 21
'''
query = {
    "query": {
        "bool": {
            "must_not": {
                "term": {
                    "name": "sally"
                }
            },
            "must": {
                "exists": {
                    "field": "age"
                }
            },
            "must": {
                "term": {
                    "age": 21
                }
            }
        }
    }
}

res = es.search(index="py_index01", body=query)

print(res)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': 'HXeZEJMB56iyadvTS7CW', '_score': 1.0, '_source': {'name': 'xiaoming', 'age': 21, 'hobbies': 'jumpping', 'city': 'bejing'}}]}}


In [19]:
# 10. bool 查询

'''
① 与 bool 过滤相似，用于合并多个查询子句。
② 不同的是，bool 过滤可以直接给出是否匹配成功， 而bool查询要计算每一个查询子句的 _score （相关性分值）
'''
# query = {
#     "query": {
#         "bool": {
#             "must": {
#                 "match": {
#                     "hobbies": "reading"
#                 }
#             },
#             "must": {
#                 "exists": {
#                     "field": "hobbies"
#                 }
#             }
#         }
#     }
# }

query = {
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "hobbies": "reading"
                    }
                },
                {
                    "exists": {
                        "field": "hobbies"
                    }
                }
            ]
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 17, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 2.112916, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 2.112916, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}]}}


In [20]:

# 11. multi_match

# match查询的基础上同时搜索多个字段，在多个字段中同时查一个
query = {
    "query": {
        "multi_match": {
            "query": "jumping",
            "fields": ["name", 'hobbies']
        }
    }
}
result = es.search(index="py_index01", body=query)
print(result)

{'took': 16, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.1129161, 'hits': [{'_index': 'py_index01', '_id': '3', '_score': 1.1129161, '_source': {'name': 'john', 'age': 22, 'city': 'beijing', 'hobbies': 'liking,biking,jumping'}}]}}


In [21]:
# 12. wildcard查询

# 使用标准的shell通配符查询
query = {
    "query": {
        "wildcard": {
            "name": "to*"
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 34, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 1.0, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}]}}


In [22]:
# 13. regexp 查询

query = {
    "query": {
        "regexp": {
            "city": ".he.*"
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 1.0, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}, {'_index': 'py_index01', '_id': '2', '_score': 1.0, '_source': {'name': 'sally', 'age': 21, 'city': 'shenzhen', 'hobbies': 'jogging,watching,running'}}]}}


In [23]:
# 14. prefix 查询

# 以什么字符开头

query = {
    "query": {
        "prefix": {
            "city": "bei"
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'py_index01', '_id': '3', '_score': 1.0, '_source': {'name': 'john', 'age': 22, 'city': 'beijing', 'hobbies': 'liking,biking,jumping'}}]}}


In [24]:
# 15. phrase match 短语匹配

# 寻找临近的几个单词
query = {
    "query": {
        "match_phrase": {
            "hobbies": "dancing"
        }
    }
}

result = es.search(index="py_index01", body=query)

print(result)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.1129161, 'hits': [{'_index': 'py_index01', '_id': '1', '_score': 1.1129161, '_source': {'name': 'tommy', 'age': 20, 'city': 'shenzhen', 'hobbies': 'singing,dancing,reading'}}]}}


In [25]:
# 16. 根据id删除数据

es.delete(index="py_index01", id=1)

ObjectApiResponse({'_index': 'py_index01', '_id': '1', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1})

In [26]:
# 17. delete_by_query 

# 删除满足条件的所有数据，查询条件必须符合DLS格式

query = {
    "query": {
        "match": {
            "name": "sally"
        }
    }
}

result =es.delete_by_query(index="py_index01", body=query)

print(result)

{'took': 40, 'timed_out': False, 'total': 1, 'deleted': 1, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


In [27]:
# 18. 删除索引

es.indices.delete(index="py_index01")

ObjectApiResponse({'acknowledged': True})

In [28]:
# 19. 检查索引是否存在

es.indices.exists(index="py_index01")

HeadApiResponse(False)

1. 全文查询（Full-text queries）
- match：进行全文搜索，适合分词字段。
- match_phrase：进行短语匹配，确保短语中的单词按顺序相邻出现。
- multi_match：在多个字段上执行 match 查询。
- common_terms：处理常用词的查询，提高长文本查询的效率。
- query_string：基于 Lucene 查询语法的高级查询，支持复杂的布尔查询和通配符。
- simple_query_string：类似于 query_string，但语法简化，更适合用户输入。
2. 精确值查询（Term-level queries）
- term：精确匹配，不分词，适用于 keyword 类型。
- terms：多个精确值匹配，相当于 OR 逻辑，适合 keyword。
- range：范围查询，用于数值和日期等连续数据。
- exists：检查字段是否存在。
- prefix：匹配字段的前缀内容。
- wildcard：通配符查询，适合小规模数据，性能较低。
- regexp：正则表达式查询，适合复杂模式匹配。
3. 复合查询（Compound queries）
- bool：组合多个查询条件，支持 must、should、must_not、filter 等子句。
- constant_score：忽略评分，返回符合条件的文档。
- dis_max：在多个查询中返回得分最高的文档。
- function_score：在查询基础上调整文档评分，用于个性化排序。
4. JOIN 查询（Joining queries）
- nested：查询嵌套类型字段。
- has_child 和 has_parent：用于父子关系查询。
- parent_id：查询具有特定父 ID 的文档。
5. 地理位置查询（Geo queries）
- geo_shape：查询特定地理形状的区域。
- geo_bounding_box：在矩形边界框内查询。
- geo_distance：查询某个点一定距离范围内的文档。
- geo_polygon：查询在指定多边形内的文档。
6. 特定用途查询（Specialized queries）
- script：基于自定义脚本的查询，用于复杂计算。
- percolate：对文档进行查询而不是反向，通常用于事先注册的查询。
- highlight：在查询结果中高亮显示匹配的字段。
- more_like_this：根据示例文档查找相似文档。
- distance_feature：基于距离特征调整得分，用于时间和地理位置的排序。
7. 聚合查询（Aggregations）
虽然不属于直接的查询类型，但在统计分析中广泛使用。常见聚合类型包括 terms、histogram、date_histogram、avg、sum、min、max、cardinality 等。